#  Data Clean Up

*  Remove unnecessary index
*  Encode Location
*  Sum and group time + location
*  High / Low Risks


# Package Imports

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#  Data dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns
import xgboost
from sklearn.model_selection import train_test_split

#  Get Data from GDrive

In [0]:
#  Authenticate Google Account for GDrive access (raw data is saved in GDrive)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Raw data file gdrive location: 1n3_bCYonqpT-me4g7v7TlO80PVbaV9HR
clean_data_downloaded = drive.CreateFile({'id': '1n3_bCYonqpT-me4g7v7TlO80PVbaV9HR'})
clean_data_downloaded.GetContentFile('cleanData2.csv')

df_cleanData = pd.read_csv('cleanData2.csv', dtype={'Ticket number': str})

#  Data Assessment

In [0]:
df_cleanData.head()

,Unnamed: 0,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday,Modified_Location
0,0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,12.0,2015,12,2015-12,Monday,WELBY WAY
1,1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,14.0,2015,12,2015-12,Monday,S MAIN ST
2,2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,20.0,2015,12,2015-12,Monday,WORLD WAY
3,3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,15.0,2015,12,2015-12,Saturday,WORLD WAY
4,4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,1.0,2015,9,2015-09,Tuesday,GEORGIA ST OLYMPIC


In [0]:
df_cleanData.dtypes

Unnamed: 0                 int64
Ticket number             object
Issue Date                object
Issue time               float64
RP State Plate            object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Issue_Hour               float64
Issue_Year                 int64
Issue_Month                int64
Issue_Month_Year          object
Issue_Weekday             object
Modified_Location         object
dtype: object

In [0]:
df_cleanData.shape

(6751096, 20)

In [0]:
#  Remove Unnecessary Index
df_cleanData.drop('Unnamed: 0', axis=1, inplace=True)
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday,Modified_Location
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,12.0,2015,12,2015-12,Monday,WELBY WAY
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,14.0,2015,12,2015-12,Monday,S MAIN ST
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,20.0,2015,12,2015-12,Monday,WORLD WAY
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,15.0,2015,12,2015-12,Saturday,WORLD WAY
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,1.0,2015,9,2015-09,Tuesday,GEORGIA ST OLYMPIC


In [0]:
df_cleanData.shape

(6751096, 19)

In [0]:
#  Encode Modified_Location
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_cleanData['Modified_Location_Encode'] = le.fit_transform(df_cleanData['Modified_Location'].astype(str))
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday,Modified_Location,Modified_Location_Encode
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,12.0,2015,12,2015-12,Monday,WELBY WAY,45303
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,14.0,2015,12,2015-12,Monday,S MAIN ST,36727
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,20.0,2015,12,2015-12,Monday,WORLD WAY,46336
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,15.0,2015,12,2015-12,Saturday,WORLD WAY,46336
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,1.0,2015,9,2015-09,Tuesday,GEORGIA ST OLYMPIC,63071


In [0]:
df_cleanData.Modified_Location_Encode.value_counts()

41454    131384
45108     45709
25730     43305
38572     38858
32227     37085
33418     34097
25352     33986
22906     31930
40234     31738
24549     31316
39843     27419
38677     27172
38557     26956
25883     26730
46005     25227
20554     25182
39645     22947
26555     22325
33412     22145
22804     22104
31380     21671
4069      21142
22582     20156
20553     19594
26760     19105
6499      18988
45712     18580
40371     17472
39838     16877
45710     16738
          ...  
63465         1
34791         1
55267         1
34793         1
75743         1
38891         1
79837         1
71641         1
83923         1
10238         1
32448         1
6136          1
26614         1
18418         1
22512         1
42990         1
47084         1
38888         1
59366         1
63460         1
75742         1
79836         1
67546         1
71640         1
83922         1
10237         1
30711         1
16072         1
11978         1
4094          1
Name: Modified_Location_

In [0]:
len(df_cleanData.Modified_Location_Encode.unique().tolist())

86704

In [0]:
len(df_cleanData.Modified_Location.unique().tolist())

86704

#  Sum and group time + location

In [0]:
df = pd.DataFrame({'Date': ['2015-05-08', '2015-05-07', '2015-05-06', '2015-05-05', '2015-05-08', '2015-05-07', '2015-05-06', '2015-05-05'], 'Sym': ['aapl', 'aapl', 'aapl', 'aapl', 'aaww', 'aaww', 'aaww', 'aaww'], 'Data2': [11, 8, 10, 15, 110, 60, 100, 40],'Data3': [5, 8, 6, 1, 50, 100, 60, 120]})

df['Data4'] = df['Data3'].groupby(df['Date']).transform('sum')

In [0]:
df.head()

,Data2,Data3,Date,Sym,Data4
0,11,5,2015-05-08,aapl,55
1,8,8,2015-05-07,aapl,108
2,10,6,2015-05-06,aapl,66
3,15,1,2015-05-05,aapl,121
4,110,50,2015-05-08,aaww,55


In [0]:
df_cleanData['Issue_Hour_Loc_Count'] = df_cleanData['Modified_Location_Encode'].groupby(df_cleanData['Issue_Hour']).transform('value_counts')

In [0]:
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,...,Violation Description,Fine amount,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday,Modified_Location,Modified_Location_Encode,Issue_Hour_Loc_Count
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,...,NO EVIDENCE OF REG,50.0,12.0,2015,12,2015-12,Monday,WELBY WAY,45303,505
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,...,NO EVIDENCE OF REG,50.0,14.0,2015,12,2015-12,Monday,S MAIN ST,36727,425
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,...,WHITE CURB,58.0,20.0,2015,12,2015-12,Monday,WORLD WAY,46336,347
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,...,17104h,NaN,15.0,2015,12,2015-12,Saturday,WORLD WAY,46336,404
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,...,NO STOPPING/STANDING,93.0,1.0,2015,9,2015-09,Tuesday,GEORGIA ST OLYMPIC,63071,1332


In [0]:
df_cleanData[df_cleanData.Modified_Location_Encode == 36727]

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,...,Violation Description,Fine amount,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday,Modified_Location,Modified_Location_Encode,Issue_Hour_Loc_Count
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,...,NO EVIDENCE OF REG,50.0,14.0,2015,12,2015-12,Monday,S MAIN ST,36727,425
47777,1111383416,2016-01-05T00:00:00,219.0,CA,MAZD,PA,GY,646 S MAIN ST,00601,56.0,...,NO PARKING,73.0,2.0,2016,1,2016-01,Tuesday,S MAIN ST,36727,1127
55789,1112951770,2016-01-04T00:00:00,1957.0,CA,TOYO,PA,WH,1804 S MAIN ST,00195,1.0,...,NO EVIDENCE OF REG,50.0,19.0,2016,1,2016-01,Monday,S MAIN ST,36727,358
77420,1109572752,2016-01-13T00:00:00,742.0,CA,BMW,PA,BL,4166 S MAIN ST,00544,55.0,...,NO STOP/STAND AM,93.0,7.0,2016,1,2016-01,Wednesday,S MAIN ST,36727,612
77421,1109572763,2016-01-13T00:00:00,743.0,CA,BMW,PA,BL,4166 S MAIN ST,00544,55.0,...,EXPIRED TAGS,25.0,7.0,2016,1,2016-01,Wednesday,S MAIN ST,36727,612
94817,1109574395,2016-01-15T00:00:00,755.0,CA,BMW,PA,BK,8324 S MAIN ST,549A,55.0,...,NO STOP/STAND AM,93.0,7.0,2016,1,2016-01,Friday,S MAIN ST,36727,612
108995,1113871441,2016-01-18T00:00:00,1410.0,CA,ACUR,PA,WH,1720 S MAIN ST,207W,51.0,...,METER EXPIRED,63.0,14.0,2016,1,2016-01,Monday,S MAIN ST,36727,425
108996,1113871452,2016-01-18T00:00:00,1412.0,CA,JEEP,PA,WH,1720 S MAIN ST,207W,51.0,...,METER EXPIRED,63.0,14.0,2016,1,2016-01,Monday,S MAIN ST,36727,425
117186,1111379006,2016-01-21T00:00:00,205.0,CA,HOND,PA,GY,807 S MAIN ST,00601,56.0,...,NO PARKING,73.0,2.0,2016,1,2016-01,Thursday,S MAIN ST,36727,1127
124191,1109008014,2016-01-19T00:00:00,1715.0,CA,TOYO,PA,GR,525 S MAIN ST,1C51,1.0,...,NO EVIDENCE OF REG,50.0,17.0,2016,1,2016-01,Tuesday,S MAIN ST,36727,380


In [0]:
df_cleanData.Issue_Hour_Loc_Count.describe()

count    6.751096e+06
mean     5.249974e+02
std      2.193471e+02
min      3.030000e+02
25%      4.040000e+02
50%      5.050000e+02
75%      5.310000e+02
max      1.856000e+03
Name: Issue_Hour_Loc_Count, dtype: float64

#  New Column:  High / Low Risks

In [0]:
def condition(s):
  if(s['Issue_Hour_Loc_Count'] > 500):
    return 1
  else:
    return 0

In [0]:
df_cleanData['t_risk_level'] = df_cleanData.apply(condition, axis=1)

In [0]:
df_cleanData.head()

,Ticket number,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Route,Agency,...,Fine amount,Issue_Hour,Issue_Year,Issue_Month,Issue_Month_Year,Issue_Weekday,Modified_Location,Modified_Location_Encode,Issue_Hour_Loc_Count,t_risk_level
0,1103341116,2015-12-21T00:00:00,1251.0,CA,HOND,PA,GY,13147 WELBY WAY,01521,1.0,...,50.0,12.0,2015,12,2015-12,Monday,WELBY WAY,45303,505,1
1,1103700150,2015-12-21T00:00:00,1435.0,CA,GMC,VN,WH,525 S MAIN ST,1C51,1.0,...,50.0,14.0,2015,12,2015-12,Monday,S MAIN ST,36727,425,0
2,1104803000,2015-12-21T00:00:00,2055.0,CA,NISS,PA,BK,200 WORLD WAY,2R2,2.0,...,58.0,20.0,2015,12,2015-12,Monday,WORLD WAY,46336,347,0
3,1104820732,2015-12-26T00:00:00,1515.0,CA,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,...,NaN,15.0,2015,12,2015-12,Saturday,WORLD WAY,46336,404,0
4,1105461453,2015-09-15T00:00:00,115.0,CA,CHEV,PA,BK,GEORGIA ST OLYMPIC,1FB70,1.0,...,93.0,1.0,2015,9,2015-09,Tuesday,GEORGIA ST OLYMPIC,63071,1332,1


#  Upload Encoded Dataframe as CSV --> gDrive


In [0]:
df_cleanData.to_csv("example3.csv")
uploaded = drive.CreateFile({'title': 'DataCleanUp.csv'})
uploaded.SetContentFile("example3.csv")
uploaded.Upload()